## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64
import pytz
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from IPython.display import HTML

from openbb_terminal.reports import widget_helpers as widgets
from openbb_terminal.sdk import openbb
from openbb_terminal.helper_funcs import get_user_timezone
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal.core.config.paths import REPOSITORY_DIRECTORY

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

In [ ]:
theme = TerminalStyle(
    os.getenv("OPENBB_MPLSTYLE") or "light",
    os.getenv("OPENBB_MPFSTYLE") or "light",
    os.getenv("OPENBB_RICHSTYLE") or "light",
)
stylesheet = widgets.html_report_stylesheet()

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
report_name = "economy_report"

In [ ]:
author = "OpenBB"
report_title = f"Economy report"
user_time_zone = pytz.timezone(get_user_timezone())
report_date = pd.Timestamp.now(tz=user_time_zone).strftime("%d %B, %Y")
report_time = pd.Timestamp.now(tz=user_time_zone).strftime("%H:%M")
report_timezone = pd.Timestamp.now(tz=user_time_zone).tzinfo
report_title, report_date, report_time, report_timezone

## Data

Summary

In [ ]:
overview = openbb.economy.overview()
overview.set_index(list(overview.columns[0]), inplace=True)
futures = openbb.economy.futures()
futures.set_index(list(futures.columns[0]), inplace=True)
news = openbb.news("economy")
news.columns = news.columns.str.title()
news.set_index("Published", inplace=True)

In [ ]:
rtps_data = openbb.economy.rtps()
fig = plt.figure(figsize=(11, 4), constrained_layout=True)
spec = fig.add_gridspec(1, 2)
ax1 = fig.add_subplot(spec[0, 1])
openbb.economy.rtps_chart(external_axes=[ax1])
ax1.set_title("")
ax1.set_ylabel("")

ax2 = fig.add_subplot(spec[0, 0])
chg = futures["%Chg"]
colors = [theme.up_color if x > 0 else theme.down_color for x in list(chg.values)]
ax2 = chg.plot(kind="barh", color=colors)
theme.style_primary_axis(ax2)
ax2.set_title("")
ax2.tick_params(axis="x", labelrotation=90)
ax2.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter("%.2f"))
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
chart_0 = f.getvalue().decode("utf-8")

Events

In [ ]:
events_1 = openbb.economy.events(
    countries="United States",
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_1.empty:
    events_1 = events_1.set_index("Date")

events_2 = openbb.economy.events(
    countries=[
        "Germany",
        "Spain",
        "Italy",
        "United Kingdom",
        "France",
        "Portugal",
        "Greece",
    ],
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_2.empty:
    events_2 = events_2.set_index("Date")

events_3 = openbb.economy.events(
    countries="China",
    start_date=datetime.datetime.now().strftime("%Y-%m-%d"),
)
if not events_3.empty:
    events_3 = events_3.set_index("Date")

Yields

In [ ]:
ycrv = openbb.economy.ycrv(return_date=True)

if (isinstance(ycrv, pd.DataFrame) and not ycrv.empty) or (
    not isinstance(ycrv, pd.DataFrame) and ycrv
):
    ycrv_data_1, _ = ycrv
    fig, (ax1) = plt.subplots(
        nrows=1,
        figsize=(16 / 9 * 5, 5),
        dpi=150,
    )
    ycrv_country_1 = "United states"

    openbb.economy.ycrv_chart(external_axes=[ax1])

    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    ycrv_chart_1 = f.getvalue().decode("utf-8")

    if not ycrv_data_1.empty:
        ycrv_data_1.drop(["Maturity"], axis=1, inplace=True)
        ycrv_data_1.index = [
            "1M",
            "3M",
            "6M",
            "1Y",
            "2Y",
            "3Y",
            "5Y",
            "7Y",
            "10Y",
            "20Y",
            "30Y",
        ]
else:
    ycrv = None

In [ ]:
openbb.economy.treasury_maturities()

In [ ]:
fig = plt.figure(figsize=(11, 7), constrained_layout=True)
spec = fig.add_gridspec(2, 2)

ax1 = fig.add_subplot(spec[0, 0])
openbb.economy.treasury_chart(
    instruments=["nominal"],
    maturities=["2y", "10y"],
    frequency="daily",
    start_date="2020-01-01",
    external_axes=[ax1],
)
ax1.set_title("UST 2y-10y", loc="left")

ax2 = fig.add_subplot(spec[0, 1])
openbb.economy.treasury_chart(
    instruments=["nominal"],
    maturities=["10y", "30y"],
    frequency="daily",
    start_date="2020-01-01",
    external_axes=[ax2],
)
ax2.set_title("UST 10y-30y", loc="left")


ax3 = fig.add_subplot(spec[1, :])
openbb.economy.treasury_chart(
    instruments=["nominal", "inflation"],
    maturities=["10y"],
    frequency="daily",
    start_date="2020-01-01",
    external_axes=[ax3],
)
ax3.set_title("Nominal vs TIPS", loc="left")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ust = f.getvalue().decode("utf-8")

Equity Market

In [ ]:
valuation = openbb.economy.valuation()
performance = openbb.economy.performance()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
data = openbb.economy.index(
    indices=["sp500", "dow_djus", "nasdaq100", "russell3000", "cboe_vix"],
    start_date="2000-03-09",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
ax.plot(data * 100)
ax.set_title("")
ax.legend(data.columns)
ax.set_ylabel("Cumulative returns (%)")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
eqty_0 = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
data = openbb.economy.index(
    indices=["ftse100", "de_dax40", "fr_cac40", "it_mib40", "es_ibex35"],
    start_date="2000-01-01",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
ax.plot(data * 100)
ax.set_title("")
ax.legend(data.columns)
ax.set_ylabel("Cumulative returns (%)")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
eqty_1 = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
data = openbb.economy.index(
    indices=["jp_n225", "cn_csi300", "hk_china", "au_asx50"], start_date="2014-01-01"
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
ax.plot(data * 100)
ax.set_title("")
ax.legend(data.columns)
ax.set_ylabel("Cumulative returns (%)")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
eqty_2 = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
data = openbb.economy.index(
    indices=["in_nse50", "br_bvsp", "za_jo", "mx_ipc", "ru_moex"],
    start_date="2013-01-01",
)

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
ax.plot(data * 100)
ax.set_title("")
ax.legend(data.columns)
ax.set_title("")
ax.set_ylabel("Cumulative returns (%)")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
eqty_3 = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
data = openbb.economy.index(indices=["arca_xoi", "arca_pse"], start_date="2000-01-01")

data = data.pct_change().dropna()
data = (1 + data.shift(periods=1, fill_value=0)).cumprod() - 1
ax.plot(data * 100)
ax.set_title("")
ax.legend(data.columns)
ax.set_title("")
ax.set_ylabel("Cumulative returns (%)")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
eqty_4 = f.getvalue().decode("utf-8")

In [ ]:
bigmac_res = openbb.economy.bigmac()

if (isinstance(bigmac_res, pd.DataFrame) and not bigmac_res.empty) or (
    not isinstance(bigmac_res, pd.DataFrame) and bigmac_res
):
    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    openbb.economy.bigmac_chart(
        country_codes=[
            "USA",
            "CAN",
            "ARG",
            "PRT",
            "NLD",
            "FRA",
            "UAE",
            "LKA",
            "VNM",
            "RUS",
            "SWE",
            "GBR",
        ],
        external_axes=[ax],
    )
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    bigmac = f.getvalue().decode("utf-8")
else:
    bigmac_res = pd.DataFrame()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
openbb.economy.macro_chart(
    parameters=["CONF", "IP"], start_date="2010-01-01", external_axes=[ax]
)
ax.set_title("")

ax1 = ax.twinx()
df, df_dict, _ = openbb.economy.macro(parameters=["CPI"], start_date="2010-01-01")
cpi_chg = df.pct_change(1) * 100
ax1.plot(cpi_chg, linestyle="--", color="red")

ax1.legend(
    ["CPI (% change)"],
    bbox_to_anchor=(0.005, 0.3, 1, -0.52),
    loc="upper left",
    borderaxespad=0,
    prop={"size": 9},
)
ax1.set_title("")

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
macro_0 = f.getvalue().decode("utf-8")

In [ ]:
fred = openbb.economy.fred(series_ids=["T10Y3M"], start_date="1980-01-01")

if (isinstance(fred, pd.DataFrame) and not fred.empty) or (
    not isinstance(fred, pd.DataFrame) and fred
):
    df, df_dict = fred

    fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
    ax.plot(df.index, df["T10Y3M"], linestyle="--", color="red")
    ax.set_ylim([-2, 6])
    ax.legend(
        [df_dict["T10Y3M"]["title"]],
        bbox_to_anchor=(0, 0.40, 1, -0.52),
        loc="upper right",
        borderaxespad=0,
        prop={"size": 9},
    )

    ax1 = ax.twinx()
    openbb.economy.fred_chart(
        series_ids=["A191RP1A027NBEA"],
        start_date="1980-01-01",
        external_axes=[ax1],
    )
    ax1.set_ylim([-3, 15])
    ax1.set_title("")

    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    fred_0 = f.getvalue().decode("utf-8")
else:
    fred = None

## Render the report template to a file

In [ ]:
body = ""

img = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/reports/templates/OpenBB_reports_logo.png"
)
floppy_disk_img = (
    str(REPOSITORY_DIRECTORY) + "/openbb_terminal/reports/templates/floppy-disc.png"
)
body += widgets.header(
    img,
    floppy_disk_img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>ECONOMY REPORT: Leverage multiple sources to create custom reports.</b>",
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Events",
        "Yields",
        "Equity Market",
        "Macro Data",
    ]
)

# Summary
htmlcode = widgets.row(
    [widgets.h(3, "Global markets (source: WSJ)") + overview.to_html()]
)

htmlcode += widgets.row(
    [widgets.h(3, "Commodities (source: WSJ)") + futures.to_html()]
    + [
        widgets.h(3, "Real-time sector performance (source: Alphavantage)")
        + rtps_data.to_html().replace("table", 'table style="display:inline"')
    ]
)
htmlcode += widgets.row([chart_0])
htmlcode += widgets.row(
    [
        widgets.h(3, "Top news on 'economy' keyword (source: Newsapi)")
        + news[:6].to_html()
    ]
)
body += widgets.add_tab("SUMMARY", htmlcode)

# Events
htmlcode = widgets.row([widgets.h(3, "Economic calendars (source: investing.com)")])
htmlcode += widgets.row([widgets.h(3, "United States") + events_1.to_html()])
htmlcode += widgets.row(
    [widgets.h(3, "Select European Countries") + events_2.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "China") + events_3.to_html()])
body += widgets.add_tab("Events", htmlcode)

# Yields
htmlcode = widgets.row([widgets.h(3, "US treasuries (source: EconDB)") + ust])

if ycrv:
    htmlcode += widgets.row(
        [widgets.h(3, f"{ycrv_country_1} yield curve (source: FRED)") + ycrv_chart_1]
    )
    htmlcode += widgets.row([widgets.h(3, "") + ycrv_data_1.to_html()])

body += widgets.add_tab("Yields", htmlcode)

# Equity market
htmlcode = widgets.row(
    [widgets.h(3, "Valuation (source: Finviz)") + valuation.to_html()]
)
htmlcode += widgets.row(
    [widgets.h(3, "Performance (source: Finviz)") + valuation.to_html()]
)
htmlcode += widgets.row([widgets.h(3, "US markets (source: YahooFinance)") + eqty_0])
htmlcode += widgets.row(
    [widgets.h(3, "European markets (source: YahooFinance)") + eqty_1]
)
htmlcode += widgets.row([widgets.h(3, "APAC markets (source: YahooFinance)") + eqty_2])
htmlcode += widgets.row(
    [widgets.h(3, "Emerging markets (source: YahooFinance)") + eqty_3]
)
htmlcode += widgets.row([widgets.h(3, "ARCA (source: YahooFinance)") + eqty_4])
body += widgets.add_tab("Equity Market", htmlcode)

# Macro
htmlcode = ""
if not bigmac_res.empty:
    htmlcode += widgets.row([widgets.h(3, "Bigmac index (source: Nasdaq)") + bigmac])
htmlcode += widgets.row([widgets.h(3, "Leading indicators (source: EconDB)") + macro_0])
if fred:
    htmlcode += widgets.row(
        [widgets.h(3, "Recession indicator (source: FRED)") + fred_0]
    )
body += widgets.add_tab("Macro Data", htmlcode)

body += widgets.tab_clickable_and_save_evt()
report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)